In [78]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torch.nn.functional as F
from torchvision import datasets, transforms
data_dir='../Datasets/Imgs'
classes=['Kirmizi_Pistachio','Siirt_Pistachio']

In [79]:
transforms = transforms.Compose([transforms.RandomRotation(30),
                                  transforms.RandomResizedCrop(224),
                                  transforms.RandomHorizontalFlip(),
                                  transforms.ToTensor(),
                                  transforms.Resize(size=(28,28))
                                ])

# Pass transforms in here, then run the next cell to see how the transforms look
train_data = datasets.ImageFolder(data_dir + '/train', transform=transforms)
test_data = datasets.ImageFolder(data_dir + '/test', transform=transforms)
val_data = datasets.ImageFolder(data_dir + '/val', transform=transforms)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=28)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=28)
val_loader = torch.utils.data.DataLoader(val_data, batch_size=28)

In [80]:
class KSOM(nn.Module):
    def __init__(self, input_dim, output_dim, learning_rate):
        super(KSOM, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=3, out_channels=12, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(12)
        self.conv2 = nn.Conv2d(in_channels=12, out_channels=24, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(24)   
        self.pool = nn.MaxPool2d(kernel_size=2)
        self.fc1 = nn.Linear(24*14*14, 2)

        self.input_dim = input_dim
        self.output_dim = output_dim
        self.learning_rate = learning_rate
        self.sigma = output_dim / 2
        # Create the weight matrix
        self.weights = nn.Parameter(torch.randn(output_dim, output_dim, input_dim))

    def forward(self, x):
        output = F.relu(self.bn1(self.conv1(x)))      
        output = F.relu(self.bn2(self.conv2(output)))     
        output = self.pool(output)                        
        # output = F.relu(self.bn4(self.conv4(output)))     
        # output = F.relu(self.bn5(self.conv5(output)))    
        output = self.fc1(output)
        # Flatten the input tensor
        # x = output.view(-1, self.input_dim)
        x = output.view( -1,24*14*14)

        # Compute the distance between each weight and the input
        distances = torch.sum((self.weights - x.unsqueeze(1)) ** 2, dim=-1)

        # Compute the winner neuron
        winner = torch.argmin(distances, dim=1)

        # Compute the neighborhood function
        neighborhood = torch.exp(-distances / (2 * self.sigma ** 2))

        # Update the weights
        delta = self.learning_rate * neighborhood.unsqueeze(-1) * (x.unsqueeze(1) - self.weights)
        self.weights.data += torch.sum(delta, dim=0)

        return winner
model = KSOM(input_dim=24*14*14, output_dim=2, learning_rate=0.1)

In [81]:
from torch.optim import Adam
 
# Define the loss function with Classification Cross-Entropy loss and an optimizer with Adam optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=0.001, weight_decay=0.0001)

In [82]:
from torch.autograd import Variable

# Function to save the model
def saveModel():
    path = "./ptModels/pist.pth"
    torch.save(model.state_dict(), path)

# Function to test the model with the test dataset and print the accuracy for the test images
def testAccuracy():
    
    model.eval()
    accuracy = 0.0
    total = 0.0
    
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            # run the model on the test set to predict labels
            outputs = model(images)
            # the label with the highest energy will be our prediction
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            accuracy += (predicted == labels).sum().item()
    
    # compute the accuracy over all test images
    accuracy = (100 * accuracy / total)
    return(accuracy)

def valAccuracy():
    
    model.eval()
    accuracy = 0.0
    total = 0.0
    
    with torch.no_grad():
        for data in val_loader:
            images, labels = data
            # run the model on the test set to predict labels
            outputs = model(images)
            # the label with the highest energy will be our prediction
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            accuracy += (predicted == labels).sum().item()
    
    # compute the accuracy over all test images
    accuracy = (100 * accuracy / total)
    return(accuracy)


# Training function. We simply have to loop over our data iterator and feed the inputs to the network and optimize.
def train(num_epochs):
    
    best_accuracy = 0.0

    # Define your execution device
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("The model will be running on", device, "device")
    # Convert model parameters and buffers to CPU or Cuda
    model.to(device)

    for epoch in range(num_epochs):  # loop over the dataset multiple times
        running_loss = 0.0

        for i, (images, labels) in enumerate(train_loader, 0):
            # get the inputs
            images = Variable(images.to(device))
            labels = Variable(labels.to(device))
            # zero the parameter gradients
            optimizer.zero_grad()
            # predict classes using images from the training set
            outputs = model(images)
            # compute the loss based on model output and real labels
            loss = loss_fn(outputs, labels)
            # backpropagate the loss
            loss.backward()
            # adjust parameters based on the calculated gradients
            optimizer.step()

            # Let's print statistics for every 1,000 images
            running_loss += loss.item()     # extract the loss value
            if i % 1000 == 999:    
                # print every 1000 (twice per epoch) 
                print('[%d, %5d] loss: %.3f' %
                      (epoch + 1, i + 1, running_loss / 1000))
                # zero the loss
                running_loss = 0.0

        # Compute and print the average accuracy fo this epoch when tested over all 10000 test images
        accuracy = testAccuracy()
        print('For epoch', epoch+1,'the test accuracy over the whole test set is %d %%' % (accuracy))
        
        # we want to save the model if the accuracy is the best
        if accuracy > best_accuracy:
            saveModel()
            best_accuracy = accuracy

In [83]:
import matplotlib.pyplot as plt
import numpy as np
batch_size = 28
# Function to show the images
def imageshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


# Function to test the model with a batch of images and show the labels predictions
def testBatch():
    # get batch of images from the test DataLoader  
    images, labels = next(iter(test_loader))

    # show all images as one image grid
    imageshow(torchvision.utils.make_grid(images))
   
    # Show the real labels on the screen 
#     print('Real labels: ', ' '.join('%5s' % classes[labels[j]] 
#                                for j in range(batch_size)))
  
    # Let's see what if the model identifiers the  labels of those example
    outputs = model(images)
    
    # We got the probability for every 10 labels. The highest (max) probability should be correct label
    _, predicted = torch.max(outputs, 1)
    
    # Let's show the predicted labels on the screen to compare with the real ones
#     print('Predicted: ', ' '.join('%5s' % classes[predicted[j]] 
#                               for j in range(batch_size)))

In [ ]:
# Function to test what classes performed well
number_of_labels = len(test_loader)
def testClassess():
    class_correct = list(0. for i in range(number_of_labels))
    class_total = list(0. for i in range(number_of_labels))
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            batch_size=len(labels)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            c = (predicted == labels).squeeze()
            for i in range(batch_size):
                label = labels[i]
                class_correct[label] += c[i].item()
                class_total[label] += 1
#     for i in range(2):
        print('Accuracy of %5s : %2d %%' % (
             classes[0],100 * class_correct[0] / class_total[0]))
#           print(class_total[i])

In [ ]:

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Let's build our model
train(5)
# print('Finished Training')

# Test which classes performed well

# Let's load the model we just created and test the accuracy per label
#     model = Network()
# path = "./pist.pth"
#     model.load_state_dict(torch.load(path))
testAccuracy()

# Test with batch of images
testBatch()
testClassess()